In [ ]:
from diffusers import DiffusionPipeline, LCMScheduler
import torch

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
lcm_lora_id = "latent-consistency/lcm-lora-sdxl"
pipe = DiffusionPipeline.from_pretrained(model_id, use_safetensors=True, variant="fp16", torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(lcm_lora_id, adapter_name="lora")
pipe.load_lora_weights("nerijs/pixel-art-xl", adapter_name="pixel")

pipe.set_adapters(["lora", "pixel"], adapter_weights=[1.0, 1.2])
pipe.enable_model_cpu_offload()

In [ ]:
from datetime import datetime
import os

prompt = "pixel, gameboy style character, 8-bit, pixel art, cute, fantasy, game character, pixelated"
negative_prompt = "3d render, realistic"

num_images = 3
output_dir = "img"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
for i in range(num_images):
    img = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=8,
        guidance_scale=1.5,
    ).images[0]
    
    img.save(f"{output_dir}/lcm_lora_{timestamp}_{i}.png")